In [3]:
import mmlspark

In [4]:
triazines = spark.read.format("libsvm")\
    .load("wasbs://publicwasb@mmlspark.blob.core.windows.net/triazines.scale.svmlight")

In [5]:
# print some basic info
print("records read: " + str(triazines.count()))
print("Schema: ")
triazines.printSchema()
triazines.limit(10).toPandas()

records read: 105
Schema: 
root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)

   label                                           features
0  0.809  (-0.6, -0.3325, -0.3325, -1.0, -1.0, -1.0, -1....
1  0.602  (-0.6, 0.0, 0.0, -1.0, -0.3325, -1.0, -1.0, 0....
2  0.442  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
3  0.718  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
4  0.697  (-0.6, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
5  0.757  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
6  0.900  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
7  0.564  (-1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1....
8  0.772  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
9  0.801  (0.2, -0.6675, -1.0, -1.0, -1.0, 0.0, -1.0, 0....
/opt/spark/python/lib/pyspark.zip/pyspark/sql/dataframe.py:2103: UserWarning: toPandas attempted Arrow optimization because 'spark.sql.execution.arrow.enabled' is set to true; however, failed by the reason below:
  Unsupported typ

In [6]:
train, test = triazines.randomSplit([0.85, 0.15], seed=1)

In [7]:
from mmlspark.lightgbm import LightGBMRegressor
model = LightGBMRegressor(objective='quantile',
                          alpha=0.2,
                          learningRate=0.3,
                          numLeaves=31).fit(train)

In [8]:
from mmlspark.lightgbm import LightGBMRegressionModel
model.saveNativeModel("mymodel")
model = LightGBMRegressionModel.loadNativeModelFromFile("mymodel")

In [9]:
print(model.getFeatureImportances())

[18.0, 4.0, 8.0, 0.0, 16.0, 16.0, 0.0, 3.0, 2.0, 0.0, 6.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 5.0, 27.0, 27.0, 18.0, 28.0, 28.0, 0.0, 10.0, 0.0, 4.0, 0.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 18.0, 0.0, 0.0, 0.0, 0.0, 0.0, 10.0, 0.0, 0.0, 0.0]

In [10]:
scoredData = model.transform(test)
scoredData.limit(10).toPandas()

label    ...     prediction
0  0.258    ...       0.414115
1  0.427    ...       0.539532
2  0.550    ...       0.537624
3  0.614    ...       0.640256
4  0.631    ...       0.422801
5  0.637    ...       0.521593
6  0.641    ...       0.585361
7  0.678    ...       0.585361
8  0.788    ...       0.726604
9  0.801    ...       0.634850

[10 rows x 3 columns]

In [11]:
from mmlspark.train import ComputeModelStatistics
metrics = ComputeModelStatistics(evaluationMetric='regression',
                                 labelCol='label',
                                 scoresCol='prediction') \
            .transform(scoredData)
metrics.toPandas()

mean_squared_error         ...           mean_absolute_error
0            0.014862         ...                      0.107673

[1 rows x 4 columns]
/home/trusted-service-user/cluster-env/env/lib/python3.6/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
  warnings.warn("pyarrow.open_stream is deprecated, please use "

In [12]:
from pyspark.ml import Transformer, Estimator, Pipeline
from pyspark.ml.classification import LogisticRegression
from mmlspark.downloader import ModelDownloader
import os, sys, time

In [14]:
model = ModelDownloader(spark, "/models/").downloadByName("ResNet50")

In [15]:
# Load the images
# use flowers_and_labels.parquet on larger cluster in order to get better results
imagesWithLabels = spark.read.parquet("wasbs://publicwasb@mmlspark.blob.core.windows.net/flowers_and_labels2.parquet") \
    .withColumnRenamed("bytes","image").sample(.1)

imagesWithLabels.printSchema()

root
 |-- filename: string (nullable = true)
 |-- image: binary (nullable = true)
 |-- labels: double (nullable = true)

In [16]:
from mmlspark.opencv import ImageTransformer
from mmlspark.image import UnrollImage, ImageFeaturizer
from mmlspark.stages import *

# Make some featurizers
it = ImageTransformer()\
    .setOutputCol("scaled")\
    .resize(height = 60, width = 60)

ur = UnrollImage()\
    .setInputCol("scaled")\
    .setOutputCol("features")
    
dc1 = DropColumns().setCols(["scaled", "image"])

lr1 = LogisticRegression().setMaxIter(8).setFeaturesCol("features").setLabelCol("labels")

dc2 = DropColumns().setCols(["features"])

basicModel = Pipeline(stages=[it, ur, dc1, lr1, dc2])

In [17]:
resnet = ImageFeaturizer()\
    .setInputCol("image")\
    .setOutputCol("features")\
    .setModelLocation(model.uri)\
    .setLayerNames(model.layerNames)\
    .setCutOutputLayers(1)
    
dc3 = DropColumns().setCols(["image"])
    
lr2 = LogisticRegression().setMaxIter(8).setFeaturesCol("features").setLabelCol("labels")

dc4 = DropColumns().setCols(["features"])

deepModel = Pipeline(stages=[resnet, dc3, lr2, dc4])

In [18]:
def timedExperiment(model, train, test):
  start = time.time()
  result =  model.fit(train).transform(test).toPandas()
  print("Experiment took {}s".format(time.time() - start))
  return result

In [19]:
train, test = imagesWithLabels.randomSplit([.8,.2])
train.count(), test.count()

(623, 164)

In [20]:
basicResults = timedExperiment(basicModel, train, test)

Experiment took 93.33845472335815s

In [21]:
deepResults = timedExperiment(deepModel, train, test)

Experiment took 179.01435899734497s

In [23]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import numpy as np

def evaluate(results, name):
    y, y_hat = results["labels"],results["prediction"]
    y = [int(l) for l in y]

    accuracy = np.mean([1. if pred==true else 0. for (pred,true) in zip(y_hat,y)])
    cm = confusion_matrix(y, y_hat)
    cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    plt.text(40, 10,"$Accuracy$ $=$ ${}\%$".format(round(accuracy*100,1)),fontsize=14)
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.colorbar()
    plt.xlabel("$Predicted$ $label$", fontsize=18)
    plt.ylabel("$True$ $Label$", fontsize=18)
    plt.title("$Normalized$ $CM$ $for$ ${}$".format(name))

plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
evaluate(deepResults,"CNTKModel + LR")
plt.subplot(1,2,2)
evaluate(basicResults,"LR")
# Note that on the larger dataset the accuracy will bump up from 44% to >90%
display(plt.show())